# form

> Main form generator that creates UI from JSON Schema.

In [ ]:
#| default_exp generators.form

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from fasthtml.common import Div, H2, Hr, FT
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui
from cjm_fasthtml_tailwind.utilities.spacing import p, m, space
from cjm_fasthtml_tailwind.utilities.sizing import w, max_w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.borders import rounded
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_jsonschema.core.parser import SchemaParser
from cjm_fasthtml_jsonschema.components.fields import create_field

In [ ]:
#| export
def generate_form_ui(
    schema: Dict[str, Any],
    values: Optional[Dict[str, Any]] = None,
    show_title: bool = True,  # Whether to show the schema title
    show_description: bool = True,  # Whether to show schema description
    compact: bool = False,  # Use compact layout (less spacing)
    card_wrapper: bool = True  # Wrap the form in a card component
) -> FT:
    """Generate a FastHTML form UI from a JSON Schema."""
    # Parse the schema
    parser = SchemaParser(schema)
    values = values or {}

    # Build form content
    content = []

    # Add title if requested
    if show_title and parser.title:
        title_component = H2(
            parser.title,
            cls=combine_classes(
                font_size._2xl,
                font_weight.bold,
                text_dui.base_content,
                m.b(2) if compact else m.b(4)
            )
        )
        if card_wrapper:
            title_component.attrs['class'] = str(card_title)
        content.append(title_component)

    # Add description if requested
    if show_description and parser.description:
        content.append(
            Div(
                parser.description,
                cls=combine_classes(
                    text_dui.base_content,
                    font_size.sm,
                    m.b(4) if compact else m.b(6)
                )
            )
        )

    # Group properties into required and optional
    required_props = parser.get_required_properties()
    optional_props = parser.get_optional_properties()

    # Add required fields
    if required_props:
        if required_props and optional_props:
            # Add section header only if we have both required and optional
            content.append(
                Div(
                    "Required Fields",
                    cls=combine_classes(
                        font_weight.semibold,
                        text_dui.base_content,
                        m.b(3)
                    )
                )
            )

        for prop in required_props:
            field_value = values.get(prop.name, prop.default)
            content.append(create_field(prop, field_value))

    # Add separator if we have both sections
    if required_props and optional_props:
        content.append(
            Hr(cls=combine_classes(m.y(4) if compact else m.y(6)))
        )

    # Add optional fields
    if optional_props:
        if required_props and optional_props:
            # Add section header only if we have both required and optional
            content.append(
                Div(
                    "Optional Fields",
                    cls=combine_classes(
                        font_weight.semibold,
                        text_dui.base_content,
                        m.b(3)
                    )
                )
            )

        for prop in optional_props:
            field_value = values.get(prop.name, prop.default)
            content.append(create_field(prop, field_value))

    # Wrap in appropriate container
    if card_wrapper:
        # Use DaisyUI card component
        return Div(
            Div(
                *content,
                cls=str(card_body)
            ),
            cls=combine_classes(
                card,
                bg_dui.base_100,
                shadow.lg,
                w.full,
                max_w._4xl
            )
        )
    else:
        # Simple div wrapper
        return Div(
            *content,
            cls=combine_classes(
                space.y(2) if compact else space.y(4),
                w.full,
                max_w._4xl,
                p(4) if compact else p(6)
            )
        )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()